# GLTF 格式教學 開篇

<a href="https://colab.research.google.com/github/CSP-GD/notes/blob/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/glb/glb.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

[`Open in observablehq`](https://observablehq.com/@toonnyy8/gltf)

## 關於 glTF

[glTF](https://www.khronos.org/gltf/) 全名為 GL Transmission Format (GL 傳輸格式)，  
是由 Khronos 所推出的新 3D 模型儲存格式，  
以容易讀取與解析的 JSON 格式作為主體，  
希望能讓開發者輕鬆、高效的存取 3D 模型。

glTF 格式實際上有三種儲存形式，分別是：
1. **.gltf + .bin + texture**  
> 由 JSON 格式的 `.gltf`、二進位格式的 `.bin` 與紋理圖片 `texture` 組合而成，  
> 並在 `.gltf` 裡使用 url 去指向 `.bin` & `texture`。
2. **.gltf only**  
> 將全部資訊都儲存在 `.gltf` 中，  
> 原本 `.bin` 與 `texture` 的部分會轉成 base64 存放於 `.gltf`。
3. **.glb**  
> 將全部資訊轉換成二進位格式儲存在 `.glb`。

## 關於本教學

本篇教學是某位萌新因心血來潮而開始編寫，  
將會用 js 一步步去解讀 glTF，  
希望能藉此將 glTF 格式推廣出去。  

> ps.  
> 實際上是某天看到 glTF 是自己能看懂的 3D 格式就很興奮的跳坑了，  
> 然後才悲劇的發現「看懂!=讀懂」，好在官方文檔還蠻完善的，  
> 網路上也能找些許的教學，所以開始非常痛苦又漫長的學習之路，  
> 期許各位讀者能從這系列教學輕鬆的了解 glTF 並完成自己的載入器。

## .glb 格式解析

作為教學的開篇就先簡單的分析 glTF 的二進位格式 glb

![圖 1. glb格式](https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/glb/gltfOverview-2.0.0b-binary.png)

圖 1. glb格式

### 先載入範例檔案

In [1]:
!wget https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/glb/cube.glb

--2020-04-06 13:19:42--  https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/glb/cube.glb
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/glb/cube.glb [following]
--2020-04-06 13:19:42--  https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/glb/cube.glb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2280 (2.2K) [application/octet-stream]
Saving to: ‘cube.glb’

cube.glb            100%[===================>

In [2]:
glb_file = open('./cube.glb', 'rb')

glb_file

<_io.BufferedReader name='./cube.glb'>

### 將檔案內容讀取出來

In [3]:
glb_bytes = glb_file.read()

glb_bytes

b'glTF\x02\x00\x00\x00\xe8\x08\x00\x00\x84\x05\x00\x00JSON{"asset":{"generator":"Khronos glTF Blender I/O v1.1.46","version":"2.0"},"scene":0,"scenes":[{"name":"Scene","nodes":[0,1,2]}],"nodes":[{"mesh":0,"name":"Cube"},{"name":"Light","rotation":[0.16907575726509094,0.7558803558349609,-0.27217137813568115,0.570947527885437],"translation":[4.076245307922363,5.903861999511719,-1.0054539442062378]},{"name":"Camera","rotation":[0.483536034822464,0.33687159419059753,-0.20870360732078552,0.7804827094078064],"translation":[7.358891487121582,4.958309173583984,6.925790786743164]}],"materials":[{"doubleSided":true,"emissiveFactor":[0,0,0],"name":"Material","pbrMetallicRoughness":{"baseColorFactor":[0.800000011920929,0.800000011920929,0.800000011920929,1],"metallicFactor":0,"roughnessFactor":0.4000000059604645}}],"meshes":[{"name":"Cube","primitives":[{"attributes":{"POSITION":0,"NORMAL":1,"TEXCOORD_0":2},"indices":3,"material":0}]}],"accessors":[{"bufferView":0,"componentType":5126,"count":24,"

### 解析 header

0~3 Bytes 為「glTF」的 ASCII 編碼

In [4]:
glb_bytes[0:4].decode()

'glTF'

第 4~7 Bytes 應該會顯示 glTF 的版本

In [5]:
gltf_version = 0

for idx, curr in enumerate(glb_bytes[4:8]):
    gltf_version += curr * 256 ** idx

gltf_version

2

第 8~11 Bytes 則代表此檔案的 Byte 數

In [6]:
print(len(glb_bytes), "Bytes")

2280 Bytes


In [7]:
gltf_length = 0

for idx, curr in enumerate(glb_bytes[8:12]):
    gltf_length += curr * 256 ** idx

gltf_length

2280

### 解析 chunk

In [0]:
import json

def chunk_parser(chunk_type, chunk_data):
    return {
        "BIN\u0000": lambda:chunk_data,
        "JSON": lambda:json.loads(chunk_data.decode()),
    }[chunk_type]()

#### chunk 0 (JSON)

In [0]:
chunk = { 'type': "", 'length': 0, 'data': "" }

由 chunk 0 的 第 0~3 Bytes 知道 chunkLength 為多少

In [10]:
chunk["length"] = 0

for idx, curr in enumerate(glb_bytes[12:12 + 4]):
    chunk["length"] += curr * 256 ** idx

chunk["length"]

1412

由 chunk 0 的 第 4~7 Bytes 知道 chunkType 為 JSON

In [11]:
chunk["type"] = glb_bytes[12 + 4:12 + 8].decode()

chunk["type"]

'JSON'

知道了 chunkType 以及 chunkLength 就可以開始解析 chunkData 了

In [12]:
chunk["data"] = chunk_parser(
  chunk["type"],
  glb_bytes[12 + 8:12 + 8 + chunk["length"]]
)

chunk["data"]

{'accessors': [{'bufferView': 0,
   'componentType': 5126,
   'count': 24,
   'max': [1, 1, 1],
   'min': [-1, -1, -1],
   'type': 'VEC3'},
  {'bufferView': 1, 'componentType': 5126, 'count': 24, 'type': 'VEC3'},
  {'bufferView': 2, 'componentType': 5126, 'count': 24, 'type': 'VEC2'},
  {'bufferView': 3, 'componentType': 5123, 'count': 36, 'type': 'SCALAR'}],
 'asset': {'generator': 'Khronos glTF Blender I/O v1.1.46', 'version': '2.0'},
 'bufferViews': [{'buffer': 0, 'byteLength': 288, 'byteOffset': 0},
  {'buffer': 0, 'byteLength': 288, 'byteOffset': 288},
  {'buffer': 0, 'byteLength': 192, 'byteOffset': 576},
  {'buffer': 0, 'byteLength': 72, 'byteOffset': 768}],
 'buffers': [{'byteLength': 840}],
 'materials': [{'doubleSided': True,
   'emissiveFactor': [0, 0, 0],
   'name': 'Material',
   'pbrMetallicRoughness': {'baseColorFactor': [0.800000011920929,
     0.800000011920929,
     0.800000011920929,
     1],
    'metallicFactor': 0,
    'roughnessFactor': 0.4000000059604645}}],
 'me

#### 剩下的 chunk 解析的方式都相同，在此就直接使用迴圈處理

In [0]:
chunks = []
offset = 12
i = 0

while(offset < len(glb_bytes)):
    chunks.append({ "type": "", "length": 0, "data": "" })

    for idx, curr in enumerate(glb_bytes[offset:offset + 4]):
        chunks[i]["length"] += curr * 256 ** idx

    chunks[i]["type"] = glb_bytes[offset + 4:offset + 8].decode()

    chunks[i]["data"] = chunk_parser(
        chunks[i]["type"],
        glb_bytes[offset + 8:offset + 8 + chunks[i]["length"]]
    )
    offset += 8 + chunks[i]["length"]
    i += 1

# chunks

最後將 chunks 整理一下

In [14]:
model = chunks[0]["data"]

for idx, buffer in enumerate(model['buffers']):
    buffer["data"] = chunks[idx + 1]["data"]

model

{'accessors': [{'bufferView': 0,
   'componentType': 5126,
   'count': 24,
   'max': [1, 1, 1],
   'min': [-1, -1, -1],
   'type': 'VEC3'},
  {'bufferView': 1, 'componentType': 5126, 'count': 24, 'type': 'VEC3'},
  {'bufferView': 2, 'componentType': 5126, 'count': 24, 'type': 'VEC2'},
  {'bufferView': 3, 'componentType': 5123, 'count': 36, 'type': 'SCALAR'}],
 'asset': {'generator': 'Khronos glTF Blender I/O v1.1.46', 'version': '2.0'},
 'bufferViews': [{'buffer': 0, 'byteLength': 288, 'byteOffset': 0},
  {'buffer': 0, 'byteLength': 288, 'byteOffset': 288},
  {'buffer': 0, 'byteLength': 192, 'byteOffset': 576},
  {'buffer': 0, 'byteLength': 72, 'byteOffset': 768}],
 'buffers': [{'byteLength': 840,
   'data': b'\x00\x00\x80?\x00\x00\x80?\x00\x00\x80?\x00\x00\x80?\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80?\x00\x00\x80

完成~~